In [ ]:
import re
import pandas as pd

In [ ]:
def preprocess(data , pattern=24):
    pattern = r"\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s"
    if pattern == 12:
        pattern = r"\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s"
    messages = re.split(pattern, data)[1:]
    dates = re.findall(pattern, data)
    df = pd.DataFrame({
        "user_message": messages,
        "message_date": dates,
    })
    df["date"] = pd.to_datetime(df["message_date"], format="%d/%m/%Y, %H:%M - ")
    users = []
    messages = []
    for message in df["user_message"]:
        # split by : and split to user , message
        entry = re.split("([\w\W]+?):\s",message)
        if entry[1:]: # if it is user message (uname : message)
            users.append(entry[1])
            messages.append(entry[2])
        else: # if it is notification chat without user name
            users.append("notification")
            messages.append(entry[0])
    df["user"] = users
    df["message"] = messages
    df.drop(columns=["user_message"] , inplace=True)

    df["only_date"] = df["date"].dt.date
    df["year"] = df['date'].dt.year
    df["month_num"] = df["date"].dt.month
    df["month"] =  df['date'].dt.month_name()
    df["day"]=  df['date'].dt.day
    df["day_name"] = df["date"].dt.day_name()
    df["hour"] = df["date"].dt.hour
    df["minute"] = df["date"].dt.minute

    period = []
    for hour in df['hour']:
        if hour == 23:
            period.append(str(hour) + "-" + str('00'))
        elif hour == 0:
            period.append(str('00') + "-" + str(hour + 1))
        else:
            period.append(str(hour) + "-" + str(hour + 1))

    df['period'] = period

    return df

In [ ]:
insomnia_group_chat = open("WhatsApp Chat with Insomniacs.txt", "r", encoding="utf-8")

In [ ]:
charan_chat  = open("WhatsApp Chat with Charan  Rkv.txt", "r", encoding="utf-8")

In [ ]:
data = charan_chat.read()
data

In [ ]:
df = preprocess(data)

In [ ]:
from datetime import timedelta
class TimeSlicer:
    def __init__(self , gap_minutes=30) -> None:
        self.gap_minutes = timedelta(minutes=gap_minutes)


    def slice(self, messages: list):
        if not messages:return "Bro nv manchi vaadiki kaadhu"
        slices = []
        current_slice = [messages[0]]
        for i in range(1 , len(messages)):
            prev_time = messages[i-1][2]
            curr_time = messages[i][2]
            time_diff = curr_time - prev_time
            if time_diff <= self.gap_minutes:
                current_slice.append(messages[i])
            else:
                slices.append(current_slice)
                current_slice = [messages[i]]
        if current_slice: slices.append(current_slice)
        return slices
    

In [ ]:
data = (list(zip(df["user"].to_list() , df["message"].to_list() , df["date"].to_list() )))
data[0]

In [ ]:
ts = TimeSlicer()

In [ ]:
slices = ts.slice(data)
[len(s) for s in slices]